In Wang et al. (2009) they estimate the divergence starting from the estimate
$$KL(p||q) \approx \frac{1}{n} \sum\limits_{i=1}^n \log \frac{\hat{p}(X_i)}{\hat{q}(X_i)}$$ 
They they estimate $\hat{p}(X_i)$ and $\hat{q}(X_i)$ using the $k$ Nearest Neighbor method to ultimately get
$$KL(p||q) \approx = \frac{d}{n} \sum\limits_{i=1}^n \log \frac{\nu_k(i)}{\rho_k(X_i)} + \log \frac{m}{n-1}$$
__However in our case, we don't need to estimate $\hat{q}(X_i)$ since $q$ in our case is just a multivariate Gaussian.__

In this notebook I will implement the new KL estimator and verify that it works for a handful of cases

In [1]:
import numpy as np
from numpy.random import multivariate_normal as mvn 
from scipy.stats import multivariate_normal as multinorm
# -- nonGaussLike -- 
from nongausslike import knn as kNN
from nongausslike import nongauss as NG

In [2]:
import matplotlib as mpl 
import matplotlib.pyplot as plt
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False
%matplotlib inline

In [ ]:
def KL_Xq(X, k): 
    n, d = X.shape # n and dimensions
    